# Energy Optimization API - Ejemplos de Uso

Este notebook demuestra cómo usar la API de predicción de consumo energético.

**Prerrequisitos:**
- API ejecutándose en `http://localhost:8000`
- Modelo entrenado disponible en `models/ensembles/`

## 1. Setup y Verificación

In [6]:
import requests
import pandas as pd
import json
from datetime import datetime

# API base URL
BASE_URL = "http://localhost:8000"

# Verificar que la API está activa
response = requests.get(f"{BASE_URL}/health")
print(f"Status: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Status: 200
{
  "status": "healthy",
  "service": "energy-optimization-api",
  "version": "1.0.0",
  "timestamp": "2025-11-06T20:41:52.005015Z",
  "model_loaded": true,
  "model_version": "stacking_ensemble_v1",
  "uptime_seconds": 569.81,
  "memory_usage_mb": 362.92,
  "cpu_usage_percent": 4.8
}


## 2. Health Check

In [7]:
# Verificar estado del servicio
response = requests.get(f"{BASE_URL}/health")
health = response.json()

print(f"Status: {health['status']}")
print(f"Model Loaded: {health['model_loaded']}")
print(f"Model Version: {health.get('model_version', 'N/A')}")
print(f"Uptime: {health['uptime_seconds']:.2f} seconds")
print(f"Memory: {health['memory_usage_mb']:.2f} MB")
print(f"CPU: {health['cpu_usage_percent']:.2f}%")

Status: healthy
Model Loaded: True
Model Version: stacking_ensemble_v1
Uptime: 600.24 seconds
Memory: 362.92 MB
CPU: 0.00%


## 3. Predicción Individual

### 3.1 Ejemplo básico

In [8]:
# Preparar datos de entrada
prediction_request = {
    "lagging_reactive_power": 23.45,
    "leading_reactive_power": 12.30,
    "co2": 0.05,
    "lagging_power_factor": 0.85,
    "leading_power_factor": 0.92,
    "nsm": 36000,  # 10:00 AM (36000 segundos desde medianoche)
    "day_of_week": 1,  # Martes
    "load_type": "Medium"
}

# Hacer predicción
response = requests.post(
    f"{BASE_URL}/predict",
    json=prediction_request
)

# Mostrar resultado
if response.status_code == 200:
    result = response.json()
    print(f" OK Predicción exitosa:")
    print(f"  Consumo predicho: {result['predicted_usage_kwh']:.2f} kWh")
    print(f"  ID: {result['prediction_id']}")
    print(f"  Modelo: {result['model_version']}")
    print(f"  Features usados: {result['features_used']}")
else:
    print(f" X Error: {response.status_code}")
    print(response.json())

✅ Predicción exitosa:
  Consumo predicho: 44.46 kWh
  ID: pred_17b22226
  Modelo: stacking_ensemble_v1
  Features usados: 9


### 3.2 Predicción para diferentes horarios

In [9]:
# Crear predicciones para diferentes horarios del día
hours = [0, 6, 12, 18]  # Medianoche, 6 AM, 12 PM, 6 PM
predictions = []

for hour in hours:
    nsm = hour * 3600  # Convertir hora a segundos desde medianoche
    
    request_data = {
        "lagging_reactive_power": 23.45,
        "leading_reactive_power": 12.30,
        "co2": 0.05,
        "lagging_power_factor": 0.85,
        "leading_power_factor": 0.92,
        "nsm": nsm,
        "day_of_week": 1,
        "load_type": "Medium"
    }
    
    response = requests.post(f"{BASE_URL}/predict", json=request_data)
    if response.status_code == 200:
        result = response.json()
        predictions.append({
            "hora": f"{hour:02d}:00",
            "consumo_kwh": result['predicted_usage_kwh']
        })

# Mostrar resultados
df_predictions = pd.DataFrame(predictions)
print("\nConsumo por horario:")
print(df_predictions.to_string(index=False))


Consumo por horario:
 hora  consumo_kwh
00:00    41.141857
06:00    45.045675
12:00    44.456209
18:00    44.456209


### 3.3 Predicción para diferentes tipos de carga

In [10]:
# Comparar consumo para diferentes tipos de carga
load_types = ["Light", "Medium", "Maximum"]
load_predictions = []

for load_type in load_types:
    request_data = {
        "lagging_reactive_power": 23.45,
        "leading_reactive_power": 12.30,
        "co2": 0.05,
        "lagging_power_factor": 0.85,
        "leading_power_factor": 0.92,
        "nsm": 36000,
        "day_of_week": 1,
        "load_type": load_type
    }
    
    response = requests.post(f"{BASE_URL}/predict", json=request_data)
    if response.status_code == 200:
        result = response.json()
        load_predictions.append({
            "tipo_carga": load_type,
            "consumo_kwh": result['predicted_usage_kwh']
        })

# Mostrar resultados
df_load = pd.DataFrame(load_predictions)
print("\nConsumo por tipo de carga:")
print(df_load.to_string(index=False))


Consumo por tipo de carga:
tipo_carga  consumo_kwh
     Light    44.456209
    Medium    44.456209
   Maximum    44.456209


## 4. Predicción Batch

Predecir para múltiples observaciones a la vez.

In [ ]:
# Crear batch de predicciones
batch_request = {
    "predictions": [
        {
            "lagging_reactive_power": 23.45,
            "leading_reactive_power": 12.30,
            "co2": 0.05,
            "lagging_power_factor": 0.85,
            "leading_power_factor": 0.92,
            "nsm": 36000,
            "day_of_week": 1,
            "load_type": "Medium"
        },
        {
            "lagging_reactive_power": 25.00,
            "leading_reactive_power": 14.00,
            "co2": 0.06,
            "lagging_power_factor": 0.80,
            "leading_power_factor": 0.90,
            "nsm": 43200,
            "day_of_week": 2,
            "load_type": "Light"
        },
        {
            "lagging_reactive_power": 30.00,
            "leading_reactive_power": 16.00,
            "co2": 0.08,
            "lagging_power_factor": 0.88,
            "leading_power_factor": 0.95,
            "nsm": 57600,
            "day_of_week": 3,
            "load_type": "Maximum"
        }
    ]
}

# Hacer predicción batch
response = requests.post(
    f"{BASE_URL}/predict/batch",
    json=batch_request
)

# Mostrar resultados
if response.status_code == 200:
    result = response.json()
    print(f"OK Predicción batch exitosa")
    print(f"\nResumen:")
    print(f"  Total predicciones: {result['summary']['total_predictions']}")
    print(f"  Consumo promedio: {result['summary']['avg_predicted_usage']:.2f} kWh")
    print(f"  Consumo mínimo: {result['summary']['min_predicted_usage']:.2f} kWh")
    print(f"  Consumo máximo: {result['summary']['max_predicted_usage']:.2f} kWh")
    print(f"  Tiempo de procesamiento: {result['summary']['processing_time_ms']:.2f} ms")
    
    print(f"\nPredicciones individuales:")
    for i, pred in enumerate(result['predictions'], 1):
        print(f"  {i}. {pred['predicted_usage_kwh']:.2f} kWh (ID: {pred['prediction_id']})")
else:
    print(f"X Error: {response.status_code}")
    print(response.json())

✅ Predicción batch exitosa

Resumen:
  Total predicciones: 3
  Consumo promedio: 34.61 kWh
  Consumo mínimo: 29.21 kWh
  Consumo máximo: 44.46 kWh
  Tiempo de procesamiento: 12.18 ms

Predicciones individuales:
  1. 44.46 kWh (ID: pred_cf5d1417)
  2. 30.17 kWh (ID: pred_e8dc28eb)
  3. 29.21 kWh (ID: pred_9485d50e)


## 5. Información del Modelo

In [12]:
# Obtener información del modelo
response = requests.get(f"{BASE_URL}/model/info")

if response.status_code == 200:
    info = response.json()
    print(f"Modelo: {info['model_name']}")
    print(f"Versión: {info['model_version']}")
    print(f"Tipo: {info['model_type']}")
    
    print(f"\nDataset de entrenamiento:")
    print(f"  Nombre: {info['training_dataset']['name']}")
    print(f"  Samples: {info['training_dataset']['samples']}")
    print(f"  Features: {info['training_dataset']['features']}")
    
    print(f"\nMétricas de entrenamiento:")
    for metric, value in info['training_metrics'].items():
        print(f"  {metric.upper()}: {value:.4f}")
    
    if info.get('base_models'):
        print(f"\nModelos base del ensemble:")
        for model in info['base_models']:
            print(f"  - {model['name']}: {model['contribution_pct']:.1f}%")
else:
    print(f"X Error: {response.status_code}")
    print(response.json())

Modelo: Stacking Ensemble
Versión: stacking_ensemble_v1
Tipo: stacking_ensemble

Dataset de entrenamiento:
  Nombre: steel_featured.parquet
  Samples: 27928
  Features: 18

Métricas de entrenamiento:
  RMSE: 12.7982
  R2: 0.8702
  MAE: 3.4731
  MAPE: 7.0100

Modelos base del ensemble:
  - XGBoost: 19.3%
  - LightGBM: 40.5%
  - CatBoost: 40.2%


## 6. Métricas del Modelo

In [13]:
# Obtener métricas actuales del modelo
response = requests.get(f"{BASE_URL}/model/metrics")

if response.status_code == 200:
    metrics = response.json()
    
    print("Métricas de producción:")
    prod = metrics['production_metrics']
    print(f"  Total predicciones: {prod['total_predictions']}")
    print(f"  Últimas 24h: {prod['predictions_last_24h']}")
    print(f"  Tiempo promedio: {prod['avg_prediction_time_ms']:.2f} ms")
    print(f"  P95 tiempo: {prod['p95_prediction_time_ms']:.2f} ms")
    print(f"  Tasa de error: {prod['error_rate_percent']:.2f}%")
    
    print(f"\nSalud del sistema:")
    health = metrics['system_health']
    print(f"  Memoria: {health['memory_usage_mb']:.2f} MB")
    print(f"  CPU: {health['cpu_usage_percent']:.2f}%")
    print(f"  Uptime: {health['uptime_seconds']:.0f} segundos")
else:
    print(f"X Error: {response.status_code}")
    print(response.json())

Métricas de producción:
  Total predicciones: 0
  Últimas 24h: 0
  Tiempo promedio: 0.00 ms
  P95 tiempo: 0.00 ms
  Tasa de error: 0.02%

Salud del sistema:
  Memoria: 256.50 MB
  CPU: 15.20%
  Uptime: 3600 segundos


## 7. Manejo de Errores

### 7.1 Validación de rangos

In [14]:
# Intentar con power factor > 1.0 (inválido)
invalid_request = {
    "lagging_reactive_power": 23.45,
    "leading_reactive_power": 12.30,
    "co2": 0.05,
    "lagging_power_factor": 1.5,  # INVÁLIDO: > 1.0
    "leading_power_factor": 0.92,
    "nsm": 36000,
    "day_of_week": 1,
    "load_type": "Medium"
}

response = requests.post(f"{BASE_URL}/predict", json=invalid_request)
print(f"Status: {response.status_code}")
if response.status_code != 200:
    error = response.json()
    print(f"Error: {error.get('error', error.get('detail'))}")
    if 'details' in error:
        for detail in error['details']:
            print(f"  Campo: {detail['field']}")
            print(f"  Mensaje: {detail['message']}")

Status: 422
Error: Validation Error
  Campo: body -> lagging_power_factor
  Mensaje: Input should be less than or equal to 1


### 7.2 Tipo de carga inválido

In [15]:
# Intentar con load_type inválido
invalid_request = {
    "lagging_reactive_power": 23.45,
    "leading_reactive_power": 12.30,
    "co2": 0.05,
    "lagging_power_factor": 0.85,
    "leading_power_factor": 0.92,
    "nsm": 36000,
    "day_of_week": 1,
    "load_type": "Invalid"  # INVÁLIDO: debe ser Light, Medium o Maximum
}

response = requests.post(f"{BASE_URL}/predict", json=invalid_request)
print(f"Status: {response.status_code}")
if response.status_code != 200:
    error = response.json()
    print(f"Error: {error.get('error', error.get('detail'))}")

Status: 422
Error: Validation Error


## 8. Uso con Python `requests`

Ejemplo de función reutilizable para hacer predicciones.

In [16]:
def predict_energy_consumption(
    lagging_reactive_power: float,
    leading_reactive_power: float,
    co2: float,
    lagging_power_factor: float,
    leading_power_factor: float,
    nsm: int,
    day_of_week: int,
    load_type: str,
    base_url: str = "http://localhost:8000"
) -> dict:
    """
    Make energy consumption prediction.
    
    Parameters
    ----------
    lagging_reactive_power : float
        Potencia reactiva en atraso (kVarh)
    leading_reactive_power : float
        Potencia reactiva en adelanto (kVarh)
    co2 : float
        Emisiones de CO2 (tCO2)
    lagging_power_factor : float
        Factor de potencia en atraso (0-1)
    leading_power_factor : float
        Factor de potencia en adelanto (0-1)
    nsm : int
        Segundos desde medianoche (0-86400)
    day_of_week : int
        Día de la semana (0=Lunes, 6=Domingo)
    load_type : str
        Tipo de carga (Light, Medium, Maximum)
    base_url : str
        API base URL
    
    Returns
    -------
    dict
        Prediction response
    
    Raises
    ------
    requests.HTTPError
        If API request fails
    """
    request_data = {
        "lagging_reactive_power": lagging_reactive_power,
        "leading_reactive_power": leading_reactive_power,
        "co2": co2,
        "lagging_power_factor": lagging_power_factor,
        "leading_power_factor": leading_power_factor,
        "nsm": nsm,
        "day_of_week": day_of_week,
        "load_type": load_type
    }
    
    response = requests.post(f"{base_url}/predict", json=request_data)
    response.raise_for_status()
    return response.json()

# Usar la función
try:
    result = predict_energy_consumption(
        lagging_reactive_power=23.45,
        leading_reactive_power=12.30,
        co2=0.05,
        lagging_power_factor=0.85,
        leading_power_factor=0.92,
        nsm=36000,
        day_of_week=1,
        load_type="Medium"
    )
    print(f"OK Consumo predicho: {result['predicted_usage_kwh']:.2f} kWh")
except requests.HTTPError as e:
    print(f"X Error: {e}")

OK Consumo predicho: 44.46 kWh


## 9. Conclusiones

Este notebook demostró:

1. ✅ Verificación del estado de la API (`/health`)
2. ✅ Predicciones individuales (`/predict`)
3. ✅ Predicciones batch (`/predict/batch`)
4. ✅ Información del modelo (`/model/info`)
5. ✅ Métricas en tiempo real (`/model/metrics`)
6. ✅ Manejo de errores y validaciones
7. ✅ Funciones reutilizables en Python
